# 1- INTRODUCTION: Gourmet Bussines

A chef has contacted me, since he needs to do a study of the businesses that have been successful in the culinary field he develops.

With respect to his experience, he has been based on Gourmet businesses in certain areas of Mexico City where they have been well received by the population.

His intention is to open a gourmet-type business in a potential place for his success and where the concept has not yet arrived.

The proposal that he makes to me is the following:

## Customer's proposal

Dear Minerva:
I am in contact with you to formally make my intention to require your artificial intelligence / machine learning service, for which I will detail below the requirements of the project you will develop.

Approach

As part of a professional growth, I intend to open a gourmet store, which will have as main selling axes:

Butchery: National and imported cuts of beef (prime quality) will be sold, wagyu meat (imported from Japan), aged meat (inside the store by us in aging chambers), pork cuts, piglets, exotic meats such as wild boar, crocodile.
Fish and seafood: Daily selection attached to sustainable fishing, fish and shellfish national and imported.
Spices, fruits, vegetables and other items related to gourmet preparations.
Workshops, courses.

Location

As part of the support I require from you, I need to determine the ideal location of the store, which could be in the Cuahutemoc or Benito Juarez delegation, for which I need to consider the following points in order to make a decision:

Purchase incidence: Take into account the purchasing trends of customers, according to nearby businesses.
Opening hours and closing of the store.
Market segmentation.

Taking these points into account, I ask for your expertise to be able to create a model and suitable with the needs that I have.

Without further ado, I remain attentive to your response.

# 2- DATA

We will rely on Foursquare to have the necessary information of the places, as well as their popularity and the business line that is of interest.

In order to locate each of the colonies in the determined areas of Mexico City, we will access the INEGI portal, which is the National Institute of Statistics and Geography of Mexico, through a web query, we can download a csv file with the geographic coordinates of all the neighborhoods of the City and to be able to work on these data.

In the question of the procedure, we can highlight the places of Gourmet turnaround related to the interest of my client and associate the types of businesses and sites that exist around it, in order to associate a set of behavior related to the gourmet sector and thus be able to extrapolate this set to a new town in the city